<a href="https://colab.research.google.com/github/dpanagop/EUvsVirus/blob/master/fake_news_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded=files.upload()

Saving news_source.xlsx to news_source.xlsx


In [3]:
!python -m spacy download fr
! pip install -U textblob
! pip install -U textblob-fr
! python -m textblob.download_corpora

     |████████████████████████████████| 14.7MB 3.8MB/s 
  Created wheel for fr-core-news-sm: filename=fr_core_news_sm-2.2.5-cp36-none-any.whl size=14727027 sha256=a4e08a5ddfb62b3641daa55d8ae43de16c45b5a6c447df2a23cba53e36f7af2e
  Stored in directory: /tmp/pip-ephem-wheel-cache-uxgmywvc/wheels/46/1b/e6/29b020e3f9420a24c3f463343afe5136aaaf955dbc9e46dfc5
Successfully built fr-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/fr_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/fr
You can now load the model via spacy.load('fr')
Requirement already up-to-date: textblob in /usr/local/lib/python3.6/dist-packages (0.15.3)
     |████████████████████████████████| 563kB 2.8MB/s 
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipp

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy
nlp = spacy.load("fr")
from textblob import TextBlob, Word
from textblob_fr import PatternTagger, PatternAnalyzer

In [8]:
flagged_words=pd.read_excel('news_source.xlsx',sheet_name='flagged_words')
word_lemma=[]
for index,row in flagged_words.iterrows():
  word=row['word']
  doc=nlp(word)
  word_lemma.append(doc[0].lemma_)
flagged_words['word_lemma']=word_lemma
flagged_words.head()

,word,type,word_lemma
0,attention,warning,attention
1,gouvernants,mention to authority,gouvernant
2,gouvernement,mention to authority,gouvernement


In [0]:
def vocabulary_check(text):
  """ A function that perfomrs the vocabulay analysis.
      
      Parameters: 
        text (str): Text to be analysed
      
    Returns: 
      result: A dataframe with columns
      -- word: the word lemma that has been detected in the vocabulary
      -- count:  the number of appearances of the word/lemma
      -- type: the type field from the vocabulary (it should explain the reason 
      for the word being in the vocabulary)
  
  """
  result={}
  doc=nlp(text)
  for token in doc:
    if token.lemma_ in flagged_words['word_lemma'].unique():
     type=flagged_words['type'][flagged_words['word_lemma']==token.lemma_].iloc[0]
     if token.lemma_ in result.keys():
        result[token.lemma_][0]=result[token.lemma_][0]+1
     else:
       result[token.lemma_]=[1,type]
  result=pd.DataFrame(result.items(),columns=['word', 'Value_list'])
  result['count']=result['Value_list'][0][0]
  result['type']=result['Value_list'][0][1]
  result=result.drop('Value_list',1)
  return(result)

class news_analysis:
  """ 
     A class that holds the results of text analysis. Espcially:
      -- polarity
      -- number of exclamation marks
      -- vocabulary based analysis
      There is also a print function that prints the results. 
  """
  polarity = np.nan
  vocabulary_datafarme=[]
  count_exclamation_marks=np.nan
  def print(self):
    print('Polarity is: ',self.polarity)
    print('Number of exclamation marks is:',self.count_exclamation_marks)
    grouped_results=self.vocabulary_datafarme.groupby('type').sum()
    for index,row in grouped_results.iterrows():
      print('Found ',row['count'],' words of type:',index)

def check_text(text,verbose=False):
  """ 
    This function takes a text in French as input and returns 
    -- its polarity
    -- an analysis based on appearance of specific keywords and symbols
  
    Parameters: 
    text (str): Text to be analysed 
    verbose (boolean): A boolean indacating whether the function will print the analysis result or not
  
    Returns: 
    analysis: A news_analysis object 
  
    """
  result=news_analysis()
  result.vocabulary_datafarme=vocabulary_check(text)
  result.count_exclamation_marks=text.count('!') 
  blob = TextBlob(text, pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())
  result.polarity=blob.polarity
  if verbose:
    result.print()
  return(result)

In [19]:
#@title Text analysis
text=" \u201CFace \xE0 la p\xE9nurie, le gouvernement nous a dit de donner des m\xE9dicaments pour chiens et chats\u201D  Mise \xE0 jour\xC0 quelques heures d'un Conseil national de s\xE9curit\xE9 qui devrait aboutir \xE0 la prolongation des actuelles mesures de confinement, plusieurs acteurs ont rappel\xE9 mercredi matin que l'heure n'est pas encore au \"d\xE9confinement\", m\xEAme si c'est une \xE9tape qui se pr\xE9pare d\xE9j\xE0. Pour \xE9viter une nouvelle flamb\xE9e, certaines mesures pourraient \xEAtre maintenues pendant plusieurs mois, jusqu'\xE0 l'arriv\xE9e d'un vaccin, selon le pr\xE9sident de l'association des syndicats m\xE9dicaux (Absym). \"Je crois que le CNS va devoir constater les faits, qui sont que l'on voit une am\xE9lioration de la situation mais que cette am\xE9lioration n'est pas suffisante. En particulier dans les unit\xE9s de soins intensifs o\xF9 l'on est presque \xE0 saturation\", a indiqu\xE9 en matin\xE9e le ministre-pr\xE9sident wallon Elio Di Rupo, interrog\xE9 par la cha\xEEne de t\xE9l\xE9vision LN24. \"On devra \xE0 coup s\xFBr maintenir les mesures de confinement au minimum jusque d\xE9but mai\", a-t-il ajout\xE9. Elles sont actuellement valables jusqu'au 19 avril (dimanche).  Des m\xE9dicaments pour chiens et chats  Intervenant en radio sur Bel RTL, le pr\xE9sident de l'Absym Philippe Devos est du m\xEAme avis. Le d\xE9confinement d\xE8s ce mois d'avril 2020, \"c'est s\xFBr que cela n'arrivera pas\", tranche le m\xE9decin, pointant la situation dans les h\xF4pitaux et une \"p\xE9nurie\" de m\xE9dicaments et s\xE9datifs qui est probl\xE9matique pour les unit\xE9s o\xF9 des patients doivent \xEAtre plac\xE9s sous respirateur. \u201CIl y a plusieurs h\xF4pitaux \xE0 qui le gouvernement a dit: \u2018on n\u2019a plus de m\xE9dicaments pour l\u2019humain donc on va vous donner des m\xE9dicaments pour chiens et chats. Ne vous tracassez pas, on a fait des tests, \xE7a devrait aller\u2019\u201D, a d\xE9nonc\xE9 le Dr Devos.  Am\xE9liorations \u201Cen mai-juin\u201D  \"En mai-juin, on pourra avoir quelque chose de progressif\", pr\xE9voit-il, mais si l'on veut \xE9viter une nouvelle flamb\xE9e de contaminations, il faudrait maintenir bien plus longtemps certaines mesures, jusqu'\xE0 l'arriv\xE9e d'un vaccin, dans \"12-18 mois\". Le fait de garder une distance d'1,5 m entre les personnes, notamment.  Vers un d\xE9confinement progressif  Un d\xE9confinement partiel et progressif devrait en tout cas d\xE9j\xE0 \xEAtre abord\xE9 par les autorit\xE9s, selon Elio Di Rupo. \"Nous allons essayer de voir d\xE8s \xE0 pr\xE9sent si on ne peut pas permettre l'ouverture de tel ou tel type de commerces, par exemple les jardineries, ou les surfaces de bricolage qui trouveraient des solutions pour que les gens ne soient pas agglutin\xE9s\", a-t-il d\xE9clar\xE9. " #@param {type:"string"}
print(text)

 “Face à la pénurie, le gouvernement nous a dit de donner des médicaments pour chiens et chats”  Mise à jourÀ quelques heures d'un Conseil national de sécurité qui devrait aboutir à la prolongation des actuelles mesures de confinement, plusieurs acteurs ont rappelé mercredi matin que l'heure n'est pas encore au "déconfinement", même si c'est une étape qui se prépare déjà. Pour éviter une nouvelle flambée, certaines mesures pourraient être maintenues pendant plusieurs mois, jusqu'à l'arrivée d'un vaccin, selon le président de l'association des syndicats médicaux (Absym). "Je crois que le CNS va devoir constater les faits, qui sont que l'on voit une amélioration de la situation mais que cette amélioration n'est pas suffisante. En particulier dans les unités de soins intensifs où l'on est presque à saturation", a indiqué en matinée le ministre-président wallon Elio Di Rupo, interrogé par la chaîne de télévision LN24. "On devra à coup sûr maintenir les mesures de confinement au minimum jus

In [24]:
analysis=check_text(text)
analysis.print()

Polarity is:  0.0
Number of exclamation marks is: 0
Found  2  words of type: mention to authority
